In [ ]:

import tensorflow as tf
print(tf.__version__)
# Additional imports
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Make the original data
we will write down a notebook for an AR model using a synthetic dataset

In [ ]:
series = np.sin(0.1*np.arange(200)) #+ np.random.randn(200)*0.1

# Plot it
plt.plot(series)


## Build the dataset
lets see if we cn use T past values to predict the next value

In [ ]:

T = 10
X = []
Y = []

for t in range(len(series) -T):
    
print("X.shape",X.shape,"Y.shape",Y.shape)
# rsults should be X.shape (190, 10) Y.shape (190,)

### Try the autoregressive linear model
    1 - create a model with one dense layer
    2 - complile model using mse loss

In [ ]:
i = ...
x = ...
model = Model(i,x)


model.compile(
    ...
)



train the RNN

In [ ]:

r = model.fit(
    X[:-N//2],Y[:-N//2],
    epochs=80,
    validation_data=(X[-N//2:],Y[-N//2:]),
)

# "Wrong" 
### forecast using true targets

In [ ]:

validation_target = Y[-N//2:]
validation_predictions = []

# index of first validation input
i = -N//2

while len(validation_predictions) < len(validation_target):
    p = # predict output  -- 1x1 array => scalar
    i += 1
    # update the predictions list
    validation_predictions.append(p)

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

### Forecast future values (use only self-predictions for making predictions)

In [ ]:

validation_target = Y[-N//2:]
validation_predictions = []

# last train input
last_x = X[-N//2] #!-D array of length T

while len(validation_predictions) < len(validation_target):
    # predict one step
    p = ...

    # update the predictions list
    validation_predictions.append(p)

    # make the new input (use np.roll)
    last_x = ...
    last_x[-1] = ...
    
plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

# Now we will use RNN

### Build the dataset
lets see if we cn use T past values to predict the next value

In [ ]:

T = 10
D = 1
X = []
Y = []
for t in range(len(series) -T):
    x = series[t:t+T]
    X.append(x)
    y = series[t+T]
    Y.append(y)

## Try the Simple RNN model

In [ ]:
from tensorflow.keras.layers import SimpleRNN

In [ ]:
X = np.array(X).reshape(-1,T,1) # now the dat should be N x T X D
Y = np.array(Y)
N = len(X)
print("X.shape",X.shape,"Y.shape",Y.shape)

# Try the Simple RNN model
i = Input(shape=(T,1))
x = SimpleRNN(6)(i)
x = Dense(1)(x)

model = Model(i,x)
model.compile(
    loss="mse",
    optimizer=Adam(lr=0.05),
)

# train the RNN
r = model.fit(
    X[:-N//2],Y[:-N//2],
    epochs=200,
    validation_data=(X[-N//2:],Y[-N//2:]),
)

In [ ]:
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])

### Forecast future values (use only self-predictions for making predictions)


In [ ]:

validation_target = Y[-N//2:]
validation_predictions = []

# last train input
last_x = X[-N//2] #!-D array of length T

while len(validation_predictions) < len(validation_target):
    p = model.predict(last_x.reshape(1,-1,1))[0,0] #1x1 array => scalar

    # update the predictions list
    validation_predictions.append(p)

    # make the new input
    last_x = np.roll(last_x,-1)
    last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

- its not perfect like AR model. 
- RNN is more flexible 
- we have some shifting

- if we set in SimpleRNN activation=None our forecast is perfect like with AR. without activation method the model reduces to a linear model

- we add noise keeping activation=None

- 1 step forecast but multistep one is bad.same for tanh activation

- with relu activation is even worse in multistep forecast. loks like copying previous value

# Things we should know and memorize

- N = number of samples
- T = sequence length
- D = number of input features
- M = number of hidden units (neurons)
- K = number of output units (neurons)

In [ ]:
# Make some data
N = 1
T = 10
D = 3
K = 2
X = np.random.randn(N, T, D)
# Make an RNN

M = 5 # number of hidden units

i = Input(shape=(T,D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)

model = Model(i,x)
# Get the output
Yhat = model.predict(X)
print(Yhat)
print('Yhat.shape', Yhat.shape)

In [ ]:
# see if we can replicate this output
# get the weights first
model.summary()
# see what's returned
model.layers[1].get_weights()
# Check their shapes
# Should make sense
# First output is input > hidden
# Second output is hidden > hidden
# Third output is bias term (vector of length M)
a,b,c = model.layers[1].get_weights()
print(a.shape,b.shape,c.shape)

# More complicated data

In [ ]:
# Additional imports
from tensorflow.keras.layers import Input,Dense,Flatten,SimpleRNN,GRU,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
# make the synthtic original data
series = np.sin((0.1*np.arange(400))**2) #x(t)=sin(ωt^2)
# plot it
plt.plot(series)

### build the dataset
# let's see if we can use T past values to predict the next value
T = 10
D = 1
X = []
Y = []
for t in range(len(series) - T):
    x = series[t:t+T]
    X.append(x)
    y = series[t+T]
    Y.append(y)

X = np.array(X).reshape(-1, T) # make it N x T
Y = np.array(Y)
N = len(X)
print("X.shape", X.shape, "Y.shape", Y.shape)



In [ ]:
# try autoregressive linear model
i = Input(shape=(T,))
x = Dense(1)(i)
model = Model(i,x)

model.compile(
    loss="mse",
    optimizer = Adam(learning_rate=0.01),
)

# train the model
r = model.fit(
    X[:-N//2], Y[:-N//2],
    epochs=80,
    validation_data=(X[-N//2:], Y[-N//2:]),
)

In [ ]:
# Forecast future values (use only self-predictions for making predictions)

validation_target = Y[-N//2:]
validation_predictions = []

# last train input
last_x = X[-N//2] #!-D array of length T

while len(validation_predictions) < len(validation_target):
    p = model.predict(last_x.reshape(1,-1))[0,0] #1x1 array => scalar

    # update the predictions list
    validation_predictions.append(p)

    # make the new input
    last_x = np.roll(last_x,-1)
    last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

### Now Try an RNN/LSTM model

In [ ]:
### Now Try an RNN/LSTM model
X = X.reshape(-1,T,1) # make it NxTxD

# make the RNN
i = Input(shape=(T,D))
x = SimpleRNN(10)(i)
x = Dense(1)(x)

model = Model(i,x)
model.compile(
    loss="mse",
    optimizer= Adam(learning_rate=0.05),
)

# train the RNN
r = model.fit(
    X[:-N//2],Y[:-N//2],
    batch_size=32,
    epochs=200,
    validation_data=(X[-N//2:],Y[-N//2:]),
)




In [ ]:
# Forecast future values (use only self-predictions for making predictions)

validation_target = Y[-N//2:]
validation_predictions = []

# last train input
last_x = X[-N//2] #!-D array of length T

while len(validation_predictions) < len(validation_target):
    p = model.predict(last_x.reshape(1,-1))[0,0] #1x1 array => scalar

    # update the predictions list
    validation_predictions.append(p)

    # make the new input
    last_x = np.roll(last_x,-1)
    last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

In [ ]:
### Now Try an RNN/LSTM model
X = X.reshape(-1,T,1) # make it NxTxD

# make the RNN
i = Input(shape=(T,D))
x = LSTM(10)(i)
x = Dense(1)(x)


model = Model(i,x)
model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=0.05),
)

# train the RNN
r = model.fit(
    X[:-N//2],Y[:-N//2],
    batch_size=32,
    epochs=200,
    validation_data=(X[-N//2:],Y[-N//2:]),
)

In [ ]:
# One-step forecast using true targets
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:,0]

plt.plot(Y, label='targets')
plt.plot(predictions, label='predictions')
plt.title('many-to-one RNN')
plt.legend()



In [ ]:
# multi-step forecast
forecast = []
input_ = X[-N//2]
while len(forecast) <len(Y[-N//2:]):
    # reshape the input to NxTxD
    f = model.predict(input_.reshape(1,T,1))[0,0]
    forecast.append(f)

    #make a new input with the latest forecast
    input_ = np.roll(input_,-1)
    input_[-1] = f

plt.plot(Y[-N//2:], label='targets')
plt.plot(forecast, label='forecast')
plt.title('RNN forecast')
plt.legend()
np.mean(np.abs(forecast-Y[-N//2:]))

In [ ]:
from tensorflow.keras.layers import GRU
### Now Try an RNN/LSTM model
X = X.reshape(-1,T,1) # make it NxTxD

# make the RNN
i = Input(shape=(T,D))
x = GRU(10)(i)
x = Dense(1)(x)


model = Model(i,x)
model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=0.05),
)

# train the RNN
r = model.fit(
    X[:-N//2],Y[:-N//2],
    batch_size=32,
    epochs=200,
    validation_data=(X[-N//2:],Y[-N//2:]),
)

In [ ]:
# One-step forecast using true targets
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:,0]

plt.plot(Y, label='targets')
plt.plot(predictions, label='predictions')
plt.title('many-to-one RNN')
plt.legend()

In [ ]:
# multi-step forecast
forecast = []
input_ = X[-N//2]
while len(forecast) <len(Y[-N//2:]):
    # reshape the input to NxTxD
    f = model.predict(input_.reshape(1,T,1))[0,0]
    forecast.append(f)

    #make a new input with the latest forecast
    input_ = np.roll(input_,-1)
    input_[-1] = f

plt.plot(Y[-N//2:], label='targets')
plt.plot(forecast, label='forecast')
plt.title('RNN forecast')
plt.legend()



In [ ]:
np.mean(np.abs(forecast-Y[-N//2:]))